In [ ]:
!pip install ftfy==6.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.0 MB/s eta 0:00:00


In [ ]:
pip install emoji nltk swifter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.6 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=60f54bfe176107a2cf27c47e17c51230b2559c2e44af6925c1c0ab7f525c7028
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


In [ ]:
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 29.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Helps support lemmatization in different languages


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import pandas as pd
import ftfy
import re
import html
import unidecode
import contractions
import emoji
import nltk
import warnings
import swifter
from bs4 import BeautifulSoup
from bs4 import MarkupResemblesLocatorWarning
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from spellchecker import SpellChecker

# Setup
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Optional: spell checker
spell = SpellChecker()
correction_cache = {}

# Load emoji dictionary
def load_emoji_dict_from_csv(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8')
    return dict(zip(df['Representation'], df['Name']))

emoji_dict = load_emoji_dict_from_csv("/content/drive/MyDrive/Research/Dataset/Fakeddit/cleaned_emojis.csv")

# Load NRC Lexicon
nrc_path = r"/content/drive/MyDrive/Research/Dataset/Fakeddit/NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
nrc_lexicon = defaultdict(set)

with open(nrc_path, "r", encoding="utf-8") as file:
    for line in file:
        word, emotion, assoc = line.strip().split('\t')
        if int(assoc) == 1:
            nrc_lexicon[word].add(emotion)

# Emotion categories
nrc_8_emotions = {"anger", "anticipation", "disgust", "fear", "joy", "sadness", "surprise", "trust"}
novelty_emotions = {"fear", "disgust", "surprise"}
expectation_emotions = {"anticipation", "sadness", "joy", "trust"}
group_map = {"expectation": 0, "neutral": 0.5, "novelty": 1}

# Cleaning & Preprocessing Functions
def fix_mojibake(text):
    return ftfy.fix_text(text) if pd.notnull(text) else text

def reduce_repeated_chars(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

def expand_contractions(text):
    return contractions.fix(text)

def correct_spelling(text):  # Optional: very slow
    corrected = []
    for word in text.split():
        if word.lower() in stop_words:
            corrected.append(word)
        else:
            if word in correction_cache:
                corrected_word = correction_cache[word]
            else:
                fixed = spell.correction(word)
                corrected_word = fixed if fixed else word
                correction_cache[word] = corrected_word
            corrected.append(corrected_word)
    return ' '.join(corrected)

def replace_emojis_custom(text, emoji_dict):
    for emoji_char, word in emoji_dict.items():
        text = text.replace(emoji_char, f" {word} ")
    return text

def clean_text_pipeline(text):
    if not isinstance(text, str): return text
    text = replace_emojis_custom(text, emoji_dict)
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = unidecode.unidecode(text)
    text = expand_contractions(text)
    text = re.sub(r"[^a-zA-Z0-9!? ]", " ", text)
    text = reduce_repeated_chars(text)
    words = [word for word in text.split() if word.lower() not in stop_words]
    text = ' '.join(words)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize(text):
    tokens = re.findall(r"\b\w+\b", str(text).lower())
    return [lemmatizer.lemmatize(token) for token in tokens]

def classify_emotion_and_group(text):
    if not isinstance(text, str) or not text.strip():
        return pd.Series(["neutral", "none", "none", 0.5])
    tokens = tokenize(text)
    emotion_count = defaultdict(int)
    for token in tokens:
        for emotion in nrc_lexicon.get(token, []):
            if emotion in nrc_8_emotions:
                emotion_count[emotion] += 1
    non_zero_emotions = {e: c for e, c in emotion_count.items() if c > 0}
    novelty_score = sum(emotion_count[e] for e in novelty_emotions)
    expectation_score = sum(emotion_count[e] for e in expectation_emotions)
    group = "neutral"
    if novelty_score > expectation_score:
        group = "novelty"
    elif expectation_score > novelty_score:
        group = "expectation"
    dominant_emotion = max(non_zero_emotions, key=non_zero_emotions.get) if non_zero_emotions else "none"
    emotion_list_str = ", ".join(sorted(non_zero_emotions.keys())) if non_zero_emotions else "none"
    return pd.Series([group, dominant_emotion, emotion_list_str, group_map[group]])

# Master Pipeline Function
def process_all_in_one(input_path, output_path, chunk_size=100000):
    first = True
    for i, chunk in enumerate(pd.read_csv(input_path, chunksize=chunk_size, encoding='latin1', low_memory=False)):
        print(f"🔄 Processing chunk {i+1}...")
        chunk['body'] = chunk['body'].swifter.apply(fix_mojibake)
        chunk['clean_body'] = chunk['body'].swifter.apply(clean_text_pipeline)
        emotion_data = chunk['clean_body'].swifter.apply(classify_emotion_and_group)
        emotion_data.columns = ['emotion_group', 'dominant_emotion', 'emotion_list', 'emotion_group_normalized']
        processed_chunk = pd.concat([chunk, emotion_data], axis=1)
        processed_chunk.to_csv(output_path, mode='w' if first else 'a', index=False, header=first, encoding='utf-8-sig')
        first = False
        print(f"✅ Finished chunk {i+1}")
        if i + 1 == 11:
            print("⏹️ Stopped after 11 chunks as requested.")
            break
    print(f"🎉 All chunks processed and saved to {output_path}")

# Run the pipeline
if __name__ == "__main__":
    input_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/all_commentsCSV.csv"
    output_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/final_emotion_processed_comments.csv"
    process_all_in_one(input_path, output_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🔄 Processing chunk 1...


Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

...

✅ Finished chunk 10
🔄 Processing chunk 11...


Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100000 [00:00<?, ?it/s]

✅ Finished chunk 11
⏹️ Stopped after 11 chunks as requested.
🎉 All chunks processed and saved to /content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/final_emotion_processed_comments.csv


In [ ]:
import pandas as pd

# Run the pipeline
if __name__ == "__main__":
    output_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/final_emotion_processed_comments.csv"
    # Load the entire DataFrame first
    df = pd.read_csv(output_path)

    # Show total rows and all column names
    print(f"Total Rows: {len(df)}")
    print("\nColumn Names:")
    print(df.columns.tolist())

    # Show sample rows
    df_sample = df.head(20)  # Display first 20 rows
    display(df_sample)


<ipython-input-11-873bfa0a8c7e>:7: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_path)


Total Rows: 1100000

Column Names:
['Unnamed: 0', 'id', 'author', 'body', 'isTopLevel', 'parent_id', 'submission_id', 'ups', 'clean_body', 'emotion_group', 'dominant_emotion', 'emotion_list', 'emotion_group_normalized']


,Unnamed: 0,id,author,body,isTopLevel,parent_id,submission_id,ups,clean_body,emotion_group,dominant_emotion,emotion_list,emotion_group_normalized
0,0,f4deplg,Haus42,"Scroll, scroll, scroll. Pause. Scroll back u...",True,t3_dkdml1,dkdml1,162.0,Scroll scroll scroll Pause Scroll back Check s...,expectation,trust,"anticipation, trust",0.0
1,1,f4d79bi,PotatoPrince84,A lot of the people who felt quite strongly ab...,True,t3_dkdml1,dkdml1,84.0,lot people felt quite strongly fake news sprea...,expectation,trust,"anticipation, fear, joy, trust",0.0
2,2,f4ddmlk,MadeToAchieveBalance,T H E S P H E R E S H A L L R I S E A ...,True,t3_dkdml1,dkdml1,34.0,H E P H E R E H L L R E G N,neutral,none,none,0.5
3,3,f4dknfn,MrBigRed,All hail the cube of justice,True,t3_dkdml1,dkdml1,9.0,hail cube justice,expectation,trust,trust,0.0
4,4,f4dgdur,burritosenior,That is glorious.,True,t3_dkdml1,dkdml1,4.0,glorious,neutral,none,none,0.5
5,5,f4exg0u,joecorey,Borg collective: origins,True,t3_dkdml1,dkdml1,3.0,Borg collective origins,neutral,none,none,0.5
6,6,f4di36l,maebeanknot,No Cube of Justice. This is Sphere of Truth land,True,t3_dkdml1,dkdml1,6.0,Cube Justice Sphere Truth land,expectation,trust,trust,0.0
7,7,f4ezvbu,Dday141,All must kneel before the cube,True,t3_dkdml1,dkdml1,2.0,must kneel cube,expectation,trust,trust,0.0
8,8,f4ilhfu,Vaffelpelten,Finally some real management,True,t3_dkdml1,dkdml1,2.0,Finally real management,expectation,trust,"anticipation, disgust, joy, surprise, trust",0.0
9,9,f4dg0vx,grezzymech,"Same. I was thinking ""it's not that crazy"". Go...",False,t1_f4deplg,dkdml1,43.0,thinking crazy Good job Onion,expectation,sadness,"anger, anticipation, fear, joy, sadness, surpr...",0.0


Combine Test, Train, validation set

In [ ]:
import pandas as pd

# Load datasets
train_df = pd.read_csv("/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/all_validate.csv")
valid_df = pd.read_csv("/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/all_train.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/all_test_public.csv")

# Align columns (optional but ensures smooth merging)
common_cols = list(set(train_df.columns) & set(valid_df.columns) & set(test_df.columns))
train_df = train_df[common_cols]
valid_df = valid_df[common_cols]
test_df  = test_df[common_cols]

# Combine all datasets
combined_df = pd.concat([train_df, valid_df, test_df], ignore_index=True)

# OPTIONAL: Drop only rows that are truly unusable
# For example, if clean_title or label is missing
required_cols = ['clean_title', '2_way_label']  # adjust as needed
combined_df = combined_df.dropna(subset=required_cols)

# Save the combined dataset
combined_df.to_csv("/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset.csv", index=False)

print("✅ Combined dataset saved. Final shape:", combined_df.shape)


✅ Combined dataset saved. Final shape: (971806, 20)


Add emotion score to combined dataset

In [ ]:
import pandas as pd

#Load comment-level emotion data
comments_df = pd.read_csv(
    "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/final_emotion_processed_comments.csv",
    low_memory=False,
    dtype=str
)

#Clean column names and convert types
comments_df.columns = comments_df.columns.str.strip()
comments_df["emotion_group_normalized"] = comments_df["emotion_group_normalized"].astype(float)

# Load main dataset
main_df = pd.read_csv(
    "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset.csv",
    low_memory=False,
    dtype=str
)

# Step 1: Group by submission_id and assign majority emotion group
def assign_emotion_group(group):
    counts = group.value_counts()
    count_0 = counts.get(0.0, 0)
    count_1 = counts.get(1.0, 0)

    if count_0 > count_1:
        return 0.0
    elif count_1 > count_0:
        return 1.0
    else:
        return 0.5

#  Compute emotion score per submission_id
emotion_scores = comments_df.groupby("submission_id")["emotion_group_normalized"].apply(assign_emotion_group).reset_index()
emotion_scores.rename(columns={"submission_id": "id", "emotion_group_normalized": "emotion_score"}, inplace=True)

#  Convert to string for merge
main_df["id"] = main_df["id"].astype(str)
emotion_scores["id"] = emotion_scores["id"].astype(str)

#Merge with main dataset
merged_df = pd.merge(main_df, emotion_scores, on="id", how="left")

#Fill missing emotion scores with -1
merged_df["emotion_score"] = merged_df["emotion_score"].fillna(-1)

# Save result
output_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset_with_emotion.csv"
merged_df.to_csv(output_path, index=False)

print("✅ Final dataset saved with emotion_score column. Shape:", merged_df.shape)


✅ Final dataset saved with emotion_score column. Shape: (971806, 21)


In [ ]:
import pandas as pd

# Run the pipeline
if __name__ == "__main__":
    output_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset_with_emotion.csv"
    # Load the entire DataFrame first
    df = pd.read_csv(output_path)

    # Show total rows and all column names
    print(f"Total Rows: {len(df)}")
    print("\nColumn Names:")
    print(df.columns.tolist())

    # Show sample rows
    df_sample = df.head(20)  # Display first 20 rows
    display(df_sample)


Total Rows: 971806

Column Names:
['Unnamed: 0.1', 'clean_title', 'id', 'Unnamed: 0.2', 'title', 'hasImage', 'upvote_ratio', 'author', '6_way_label', 'Unnamed: 0.1.1', 'created_utc', 'score', 'domain', 'image_url', 'linked_submission_id', '3_way_label', '2_way_label', 'subreddit', 'num_comments', 'Unnamed: 0', 'emotion_score']


,Unnamed: 0.1,clean_title,id,Unnamed: 0.2,title,hasImage,upvote_ratio,author,6_way_label,Unnamed: 0.1.1,...,score,domain,image_url,linked_submission_id,3_way_label,2_way_label,subreddit,num_comments,Unnamed: 0,emotion_score
0,NaN,my xbox controller says hi,cypw96,1,My Xbox controller says hi,True,0.72,singingdart7854,0,NaN,...,25,i.redd.it,https://preview.redd.it/l0ga0tug17k31.jpg?widt...,NaN,0,1,mildlyinteresting,4.0,1,-1.0
1,NaN,new image from the mandalorian,d0bzlq,2,PsBattle: New image from The Mandalorian,True,0.92,mandal0re,0,NaN,...,21,i.imgur.com,https://external-preview.redd.it/VX7bXDu9Gl8UZ...,NaN,0,1,photoshopbattles,5.0,2,0.0
2,126554.0,say hello to my little friend,d2ezoob,3,Say hello to my little friend!,True,NaN,HE_WHO_DRUELS,4,582609.0,...,10,NaN,http://i.imgur.com/F1Zbl3D.jpg,4g6bp9,2,0,psbattle_artwork,NaN,3,-1.0
3,228704.0,watch your step little one,cjqctpw,4,Watch your step little one,True,NaN,eNaRDe,4,1014196.0,...,1,NaN,http://i.imgur.com/KRyMjn1.jpg,2diyh3,2,0,psbattle_artwork,NaN,4,-1.0
4,NaN,this tree i found with a solo cup on it,bq3yuk,5,This tree I found with a solo cup on it,True,0.62,Thebubster2001,0,NaN,...,6,i.redd.it,https://preview.redd.it/bxp58zf01zy21.jpg?widt...,NaN,0,1,mildlyinteresting,8.0,5,-1.0
5,NaN,the use of space with this drawer,cpr3td,6,The use of space with this drawer.,False,0.97,Oliver9191,0,NaN,...,105,i.redd.it,NaN,NaN,0,1,mildlyinteresting,17.0,6,-1.0
6,14930.0,this cat out of a box,en2chek,7,this cat out of a box,True,NaN,JJ935,4,72124.0,...,1,NaN,https://i.imgur.com/ApENEGk.jpg,bn4kjy,2,0,psbattle_artwork,NaN,7,-1.0
7,NaN,viboots she knows you know,8dwmvd,8,ViBoots - She Knows You Know,False,0.75,NaN,1,NaN,...,2,i.redd.it,NaN,NaN,2,0,fakealbumcovers,2.0,8,-1.0
8,NaN,dude id feel the same if i got a pole through ...,8rsoq0,9,"Dude, I'd feel the same if I got a pole throug...",True,0.79,jokkstermokkster,2,NaN,...,7,i.redd.it,https://preview.redd.it/ipfrolruml411.jpg?widt...,NaN,2,0,pareidolia,0.0,9,-1.0
9,279996.0,is it time for oktoberfest already,ccewep5,10,"Is it time for Oktoberfest, already?",True,NaN,patch5,4,1219048.0,...,2,NaN,http://i.imgur.com/GNoPRww.jpg,1n28q8,2,0,psbattle_artwork,NaN,10,-1.0


In [ ]:
import pandas as pd

# Load the dataset with emotion scores
file_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset_with_emotion.csv"
df = pd.read_csv(file_path)

# Convert 'emotion_score' column to float (if not already)
df['emotion_score'] = df['emotion_score'].astype(float)

# Count posts by emotion_score values
count_1 = (df['emotion_score'] == 1).sum()
count_0_5 = (df['emotion_score'] == 0.5).sum()
count_0 = (df['emotion_score'] == 0).sum()
count_neg1 = (df['emotion_score'] == -1).sum()

print(f"Count of posts with emotion_score = 1: {count_1}")
print(f"Count of posts with emotion_score = 0.5: {count_0_5}")
print(f"Count of posts with emotion_score = 0: {count_0}")
print(f"Count of posts with emotion_score = -1: {count_neg1}")


Count of posts with emotion_score = 1: 3696
Count of posts with emotion_score = 0.5: 12380
Count of posts with emotion_score = 0: 90644
Count of posts with emotion_score = -1: 865086


In [ ]:
import pandas as pd

# Load the dataset from the CSV file
file_path = "/content/drive/MyDrive/Research/Dataset/Fakeddit/all_samples (also includes non multimodal)/Original dataset/cleaned_combined_dataset.csv"
df = pd.read_csv(file_path)

# Check the column names to be sure '2_way_label' exists
print(df.columns)

# Count how many samples have 0 and 1 in '2_way_label'
count_0 = (df['2_way_label'] == 0).sum()
count_1 = (df['2_way_label'] == 1).sum()

print(f"Number of samples with 2_way_label = 0: {count_0}")
print(f"Number of samples with 2_way_label = 1: {count_1}")

# Or just use value_counts for a quick summary:
print("\nValue counts for '2_way_label':")
print(df['2_way_label'].value_counts())


Index(['Unnamed: 0.1', 'clean_title', 'id', 'Unnamed: 0.2', 'title',
       'hasImage', 'upvote_ratio', 'author', '6_way_label', 'Unnamed: 0.1.1',
       'created_utc', 'score', 'domain', 'image_url', 'linked_submission_id',
       '3_way_label', '2_way_label', 'subreddit', 'num_comments',
       'Unnamed: 0'],
      dtype='object')
Number of samples with 2_way_label = 0: 487085
Number of samples with 2_way_label = 1: 484721

Value counts for '2_way_label':
2_way_label
0    487085
1    484721
Name: count, dtype: int64
